In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# CELL 1: DIGITAL ORGANISM - INFINITE TRAINING ENGINE
# Copy this entire cell into your notebook as the first cell

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import shutil
import random
from datetime import datetime

# ==================== CONFIGURATION ====================
SEED = 42
CHANNELS = 16
GRID_SIZE = 72
BATCH_SIZE = 8
LEARNING_RATE = 1e-3
STEPS_PER_LIFE = 96
SAVE_INTERVAL = 100  # Save every N epochs
CRYO_PATH = "digital_organism_save"

# ==================== INITIALIZATION ====================
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs(CRYO_PATH, exist_ok=True)

print(f"🧬 Digital Organism Training System")
print(f"   Device: {device}")
print(f"   Architecture: {CHANNELS}D Neural Automata on {GRID_SIZE}x{GRID_SIZE} grid")
print(f"   Save Path: {CRYO_PATH}/")

# ==================== THE GENOME ====================
class OrganismDNA(nn.Module):
    """The neural rules that govern cell behavior"""
    def __init__(self, channels=16, hidden=128):
        super().__init__()
        self.perceive = nn.Conv2d(channels, hidden, 3, padding=1)
        self.think = nn.Conv2d(hidden, channels, 1)

        # Start from dormant state
        nn.init.zeros_(self.think.weight)
        nn.init.zeros_(self.think.bias)

    def forward(self, x):
        perception = torch.relu(self.perceive(x))
        update = self.think(perception)

        # Stochastic firing (life is not deterministic)
        b, c, h, w = x.shape
        mask = (torch.rand(b, 1, h, w, device=x.device) > 0.5).float()

        return x + update * mask

# ==================== ENVIRONMENT ====================
def make_seed(batch_size, channels=CHANNELS, size=GRID_SIZE):
    """Birth - single active cell in center"""
    x = torch.zeros(batch_size, channels, size, size, device=device)
    mid = size // 2
    x[:, :, mid, mid] = 1.0
    return x

def inject_context(x, ctx_vector):
    """Inject 3D context signal into last 3 channels"""
    b, c, h, w = x.shape
    ctx = [torch.full((b, 1, h, w), float(v), device=device) for v in ctx_vector]
    return torch.cat([x[:, :-3, :, :]] + ctx, dim=1)

def get_target(context, step):
    """Generate dynamic targets based on context"""
    Y, X = torch.meshgrid(
        torch.arange(GRID_SIZE, device=device),
        torch.arange(GRID_SIZE, device=device),
        indexing='ij'
    )

    # Context: [complexity, movement_speed, size]
    complexity, speed, size = context

    # Dynamic movement
    angle = step * (0.05 + speed * 0.15)
    radius = 12.0 * (1.0 + size * 0.5)
    cx = (GRID_SIZE // 2) + radius * np.cos(angle)
    cy = (GRID_SIZE // 2) + radius * np.sin(angle)

    dist = torch.sqrt((X - cx)**2 + (Y - cy)**2)

    # Shape complexity
    pattern_size = 8.0 * (1.0 + complexity)
    target = torch.exp(-(dist**2) / (2 * pattern_size**2))

    return target.unsqueeze(0).unsqueeze(0).repeat(BATCH_SIZE, 4, 1, 1)

def damage_organism(x, severity=0.3):
    """Random damage for robustness testing"""
    b, c, h, w = x.shape
    mask = torch.ones_like(x)

    for i in range(b):
        if random.random() < severity:
            cx = random.randint(h//4, 3*h//4)
            cy = random.randint(w//4, 3*w//4)
            radius = random.randint(5, 12)

            Y, X = torch.meshgrid(
                torch.arange(h, device=device),
                torch.arange(w, device=device),
                indexing='ij'
            )
            dist = torch.sqrt((X - cx)**2 + (Y - cy)**2)
            mask[i, :, :, :] *= (dist > radius).float()

    return x * mask

# ==================== PERSISTENCE ====================
class CryoStorage:
    """Save/Load system for eternal training"""

    def freeze(self, dna, optimizer, epoch, loss_history, metadata=None):
        """Save complete organism state"""
        filepath = os.path.join(CRYO_PATH, f"organism_epoch_{epoch}.pth")

        capsule = {
            'dna': dna.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch,
            'loss_history': loss_history,
            'rng_state': torch.get_rng_state(),
            'metadata': metadata or {},
            'timestamp': datetime.now().isoformat()
        }

        torch.save(capsule, filepath)

        # Keep a "latest" for easy resumption
        latest_path = os.path.join(CRYO_PATH, "organism_latest.pth")
        shutil.copy(filepath, latest_path)

        # Create downloadable zip
        shutil.make_archive("digital_organism_backup", 'zip', CRYO_PATH)

        print(f"💾 Saved: Epoch {epoch} | {filepath}")
        return filepath

    def resurrect(self, dna, optimizer):
        """Load organism from disk"""
        filepath = os.path.join(CRYO_PATH, "organism_latest.pth")

        if not os.path.exists(filepath):
            print("🌱 No save found. Starting from Genesis.")
            return 0, []

        print(f"⚡ Resurrecting from {filepath}...")
        capsule = torch.load(filepath, map_location=device)

        dna.load_state_dict(capsule['dna'])
        optimizer.load_state_dict(capsule['optimizer'])
        torch.set_rng_state(capsule['rng_state'].cpu())

        epoch = capsule['epoch']
        loss_history = capsule['loss_history']

        print(f"✅ Restored: Epoch {epoch} | Age: {len(loss_history)} generations")
        return epoch + 1, loss_history

# ==================== TRAINING ENGINE ====================
organism = OrganismDNA(CHANNELS).to(device)
optimizer = torch.optim.Adam(organism.parameters(), lr=LEARNING_RATE)
cryo = CryoStorage()

# Try to load existing organism
start_epoch, loss_log = cryo.resurrect(organism, optimizer)

print(f"\n{'='*60}")
print(f"🚀 TRAINING INITIALIZED")
print(f"   Starting Epoch: {start_epoch}")
print(f"   Parameters: {sum(p.numel() for p in organism.parameters()):,}")
print(f"   Press Interrupt to pause/save anytime")
print(f"{'='*60}\n")

# ==================== INFINITE TRAINING LOOP ====================
try:
    epoch = start_epoch

    while True:  # Train forever until interrupted
        optimizer.zero_grad()

        # Random context (enables diverse behaviors)
        context = [random.random() for _ in range(3)]

        # Birth
        x = make_seed(BATCH_SIZE)

        loss_accum = 0.0

        # Lifetime simulation
        for step in range(STEPS_PER_LIFE):
            # Inject context
            x = inject_context(x, context)

            # Live one step
            x = organism(x)

            # Random damage (test robustness)
            if step == STEPS_PER_LIFE // 2 and random.random() < 0.3:
                x = damage_organism(x, severity=0.2)

            # Evaluate (sample to save compute)
            if step > 20 and step % 15 == 0:
                target = get_target(context, step)
                loss = F.mse_loss(x[:, :4, :, :], target)
                loss_accum += loss

        # Evolution step
        final_loss = loss_accum / ((STEPS_PER_LIFE - 20) // 15)
        final_loss.backward()
        torch.nn.utils.clip_grad_norm_(organism.parameters(), 1.0)
        optimizer.step()

        loss_log.append(final_loss.item())

        # Reporting
        if epoch % 10 == 0:
            avg_recent = np.mean(loss_log[-100:]) if len(loss_log) > 100 else np.mean(loss_log)
            print(f"Epoch {epoch:6d} | Loss: {final_loss.item():.5f} | Avg(100): {avg_recent:.5f} | Activity: {x[:, :4].abs().mean():.4f}")

        # Auto-save
        if epoch % SAVE_INTERVAL == 0 and epoch > 0:
            cryo.freeze(organism, optimizer, epoch, loss_log,
                       metadata={'avg_loss': np.mean(loss_log[-1000:])})

        epoch += 1

except KeyboardInterrupt:
    print("\n🛑 Training interrupted by user")

finally:
    # Emergency save
    print("\n💾 Saving final state...")
    cryo.freeze(organism, optimizer, epoch, loss_log)
    print("✅ Complete. Safe to disconnect.")
    print(f"📦 Download: digital_organism_backup.zip")
    print(f"   Contains: All checkpoints + latest state")
    print(f"   Resume anytime by uploading and running this cell again!")

In [ ]:
# CELL 2: VISUALIZATION & INTERACTIVE DASHBOARD
# Run this in a separate cell after Cell 1 completes

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import animation
from IPython.display import display, Image, HTML
import ipywidgets as widgets
from collections import deque
import io
import time
import threading
from math import pi

# ==================== SETUP ====================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CRYO_PATH = "digital_organism_save"

print("🔬 Initializing Visualization Dashboard...")

# ==================== ARCHITECTURE RECONSTRUCTION ====================
class OrganismDNA(nn.Module):
    def __init__(self, channels=16, hidden=128):
        super().__init__()
        self.perceive = nn.Conv2d(channels, hidden, 3, padding=1)
        self.think = nn.Conv2d(hidden, channels, 1)

    def forward(self, x):
        perception = torch.relu(self.perceive(x))
        update = self.think(perception)
        b, c, h, w = x.shape
        mask = (torch.rand(b, 1, h, w, device=x.device) > 0.5).float()
        return x + update * mask

# Load organism
model = OrganismDNA(16, 128).to(device)
try:
    checkpoint = torch.load(f"{CRYO_PATH}/organism_latest.pth", map_location=device)
    model.load_state_dict(checkpoint['dna'])
    model.eval()
    print(f"✅ Loaded organism from Epoch {checkpoint['epoch']}")
except:
    print("⚠️ No trained organism found. Using fresh DNA.")

# ==================== METRICS ENGINE ====================
class MetricsTracker:
    def __init__(self, history=200):
        self.entropy = deque(maxlen=history)
        self.coherence = deque(maxlen=history)
        self.complexity = deque(maxlen=history)
        self.energy = deque(maxlen=history)
        self.variance = deque(maxlen=history)

    def calculate_entropy(self, x):
        p = x.abs()
        p = p / (p.sum() + 1e-8)
        return -(p * torch.log(p + 1e-8)).sum().item()

    def calculate_coherence(self, x):
        fft = torch.fft.fft2(x)
        power = torch.abs(fft)
        return (power.max() / (power.mean() + 1e-8)).item()

    def calculate_complexity(self, x):
        unique = len(torch.unique(x.round(decimals=2)))
        return unique / x.numel()

    def update(self, x):
        brain = x[0, 4, :, :]
        self.entropy.append(self.calculate_entropy(brain))
        self.coherence.append(self.calculate_coherence(brain))
        self.complexity.append(self.calculate_complexity(brain))
        self.energy.append(x[0, :13].abs().sum().item())
        self.variance.append(brain.var().item())

    def get_current(self):
        return {
            'entropy': self.entropy[-1] if self.entropy else 0,
            'coherence': self.coherence[-1] if self.coherence else 0,
            'complexity': self.complexity[-1] if self.complexity else 0,
            'energy': self.energy[-1] if self.energy else 0,
            'variance': self.variance[-1] if self.variance else 0
        }

metrics = MetricsTracker()

# ==================== ENVIRONMENT ====================
def make_seed():
    x = torch.zeros(1, 16, 72, 72, device=device)
    x[:, :, 36, 36] = 1.0
    return x

def inject_context(x, ctx):
    b, c, h, w = x.shape
    layers = [torch.full((b, 1, h, w), float(v), device=device) for v in ctx]
    return torch.cat([x[:, :-3]] + layers, dim=1)

# ==================== STATUS GENERATOR ====================
def generate_status(ctx, metrics):
    c1, c2, c3 = ctx

    # Energy state
    states = ["Dormant", "Awakening", "Active", "Energized", "Hypercritical", "SUPERNOVA"]
    energy = states[min(int(c1 * 6), 5)]

    # Structure
    structs = ["Quantum", "Fluid", "Forming", "Structured", "Crystalline", "Diamond"]
    struct = structs[min(int(c2 * 6), 5)]

    # Mind state
    minds = ["Void", "Zen", "Alert", "Focused", "Intense", "TRANSCENDENT"]
    mind = minds[min(int(c3 * 6), 5)]

    ent = metrics['entropy']
    emergence = "Simple" if ent < 2 else "Complex" if ent < 4 else "INFINITE"

    return f"{energy} | {struct} | {mind}\n{emergence} Pattern"

# ==================== DASHBOARD RENDERER ====================
def render_dashboard(x, step, ctx, temp, chaos, metrics_data):
    img_body = x[0, :3].permute(1, 2, 0).cpu().clamp(0, 1).numpy()
    img_brain = x[0, 4].cpu().numpy()
    img_memory = x[0, 5:8].permute(1, 2, 0).cpu().clamp(0, 1).numpy()
    img_soul = x[0, 8].cpu().numpy()

    plt.style.use('dark_background')
    fig = plt.figure(figsize=(20, 11), facecolor='#0a0a0a')
    gs = GridSpec(4, 6, figure=fig, hspace=0.3, wspace=0.3)

    # Main body view
    ax_body = fig.add_subplot(gs[:2, :2])
    ax_body.imshow(img_body)
    status = generate_status(ctx, metrics_data)
    ax_body.set_title(f"🌌 ORGANISM STATUS\n{status}",
                     color='cyan', fontsize=16, fontweight='bold', pad=20)
    ax_body.axis('off')

    # Brain scan
    ax_brain = fig.add_subplot(gs[0, 2])
    im = ax_brain.imshow(img_brain, cmap='magma', interpolation='bilinear')
    ax_brain.set_title("🧠 NEURAL ACTIVITY", color='orange', fontsize=11)
    ax_brain.axis('off')
    plt.colorbar(im, ax=ax_brain, fraction=0.046)

    # Memory matrix
    ax_mem = fig.add_subplot(gs[0, 3])
    ax_mem.imshow(img_memory)
    ax_mem.set_title("💾 MEMORY STATE", color='lime', fontsize=11)
    ax_mem.axis('off')

    # Soul energy
    ax_soul = fig.add_subplot(gs[1, 2])
    ax_soul.imshow(img_soul, cmap='twilight')
    ax_soul.set_title("✨ LIFE FORCE", color='violet', fontsize=11)
    ax_soul.axis('off')

    # Context radar
    ax_radar = fig.add_subplot(gs[1, 3], polar=True)
    categories = ['Context 1', 'Context 2', 'Context 3', 'Chaos', 'Temp/3']
    values = list(ctx) + [chaos, temp/3]
    angles = np.linspace(0, 2*pi, len(categories), endpoint=False).tolist()
    values += values[:1]
    angles += angles[:1]

    ax_radar.fill(angles, values, color='lime', alpha=0.25)
    ax_radar.plot(angles, values, color='lime', linewidth=2, marker='o')
    ax_radar.set_ylim(0, 1)
    ax_radar.set_xticks(angles[:-1])
    ax_radar.set_xticklabels(categories, color='white', size=8)
    ax_radar.set_title("⚗️ PARAMETERS", color='lime', fontsize=11, pad=15)
    ax_radar.grid(True, alpha=0.3)

    # Entropy timeline
    ax_ent = fig.add_subplot(gs[2, :2])
    if len(metrics.entropy) > 1:
        ax_ent.plot(list(metrics.entropy), color='magenta', linewidth=2, alpha=0.8)
        ax_ent.fill_between(range(len(metrics.entropy)), list(metrics.entropy),
                           alpha=0.3, color='magenta')
    ax_ent.set_title("📊 ENTROPY (Freedom)", color='magenta', fontsize=11)
    ax_ent.set_ylim(0, 6)
    ax_ent.grid(True, alpha=0.2)
    ax_ent.set_ylabel('Entropy', color='white')

    # Coherence timeline
    ax_coh = fig.add_subplot(gs[2, 2:4])
    if len(metrics.coherence) > 1:
        ax_coh.plot(list(metrics.coherence), color='cyan', linewidth=2, alpha=0.8)
    ax_coh.set_title("🎯 COHERENCE", color='cyan', fontsize=11)
    ax_coh.grid(True, alpha=0.2)
    ax_coh.set_ylabel('Coherence', color='white')

    # Energy timeline
    ax_nrg = fig.add_subplot(gs[3, :2])
    if len(metrics.energy) > 1:
        ax_nrg.plot(list(metrics.energy), color='red', linewidth=2, alpha=0.8)
    ax_nrg.set_title("⚡ ENERGY", color='red', fontsize=11)
    ax_nrg.grid(True, alpha=0.2)
    ax_nrg.set_ylabel('Energy', color='white')
    ax_nrg.set_xlabel('Time', color='white')

    # Complexity timeline
    ax_comp = fig.add_subplot(gs[3, 2:4])
    if len(metrics.complexity) > 1:
        ax_comp.plot(list(metrics.complexity), color='yellow', linewidth=2, alpha=0.8)
    ax_comp.set_title("🧩 COMPLEXITY", color='yellow', fontsize=11)
    ax_comp.grid(True, alpha=0.2)
    ax_comp.set_ylabel('Complexity', color='white')
    ax_comp.set_xlabel('Time', color='white')

    # Metrics panel
    ax_metrics = fig.add_subplot(gs[:2, 4:])
    ax_metrics.axis('off')

    metrics_text = f"""
╔═══════════════════════════════════╗
║    🎯 QUANTUM METRICS PANEL       ║
╚═══════════════════════════════════╝

⏱️  STEP: {step:,}

📊 ENTROPY:      {metrics_data['entropy']:.3f}
📈 VARIANCE:     {metrics_data['variance']:.3f}
🎯 COHERENCE:    {metrics_data['coherence']:.3f}
🧩 COMPLEXITY:   {metrics_data['complexity']:.3f}
⚡ ENERGY:       {metrics_data['energy']:.1f}

🔵 CONTEXT 1:    {ctx[0]:.2f}
🟢 CONTEXT 2:    {ctx[1]:.2f}
🟡 CONTEXT 3:    {ctx[2]:.2f}
🌡️ TEMPERATURE:  {temp:.2f}
🌀 CHAOS:        {chaos:.2f}

╔═══════════════════════════════════╗
║    SYSTEM STATUS: OPERATIONAL     ║
╚═══════════════════════════════════╝
    """

    ax_metrics.text(0.1, 0.95, metrics_text, transform=ax_metrics.transAxes,
                   fontsize=11, verticalalignment='top', family='monospace',
                   color='lime', bbox=dict(boxstyle='round', facecolor='black', alpha=0.8))

    # Spectral analysis
    ax_spec = fig.add_subplot(gs[2:, 4:])
    fft = torch.fft.fft2(x[0, 4])
    power = torch.abs(torch.fft.fftshift(fft)).cpu().numpy()
    ax_spec.imshow(np.log(power + 1), cmap='plasma')
    ax_spec.set_title("🌈 SPECTRAL ANALYSIS", color='white', fontsize=11)
    ax_spec.axis('off')

    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', facecolor='#0a0a0a', dpi=100)
    plt.close(fig)
    return buf.getvalue()

# ==================== INTERACTIVE CONTROLS ====================
style = {'description_width': '120px'}
layout = widgets.Layout(width='30%', margin='5px')

s_ctx1 = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01,
                             description='🔵 Context 1', style=style, layout=layout)
s_ctx2 = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01,
                             description='🟢 Context 2', style=style, layout=layout)
s_ctx3 = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01,
                             description='🟡 Context 3', style=style, layout=layout)
s_temp = widgets.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1,
                            description='🌡️ Temperature', style=style, layout=layout)
s_chaos = widgets.FloatSlider(value=0.0, min=0, max=1, step=0.01,
                             description='🌀 Chaos', style=style, layout=layout)
s_speed = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1,
                             description='⚡ Speed', style=style, layout=layout)

btn_pause = widgets.ToggleButton(value=False, description='⏸️ PAUSE',
                                button_style='warning', layout=widgets.Layout(width='150px'))
btn_reset = widgets.Button(description='🔄 RESET', button_style='info',
                          layout=widgets.Layout(width='150px'))
btn_snapshot = widgets.Button(description='📸 SNAPSHOT', button_style='success',
                             layout=widgets.Layout(width='150px'))
btn_stop = widgets.Button(description='🛑 STOP', button_style='danger',
                         layout=widgets.Layout(width='150px'))

screen = widgets.Image(format='png', width=1400, height=800)
output = widgets.Output()

# ==================== CONTROLLER ====================
class SimulationController:
    def __init__(self):
        self.running = False
        self.paused = False
        self.x = None
        self.step = 0

    def reset_state(self):
        self.x = make_seed()
        self.step = 0
        metrics.entropy.clear()
        metrics.coherence.clear()
        metrics.complexity.clear()
        metrics.energy.clear()
        metrics.variance.clear()

    def update_step(self):
        if self.paused:
            return

        ctx = [s_ctx1.value, s_ctx2.value, s_ctx3.value]
        temp = s_temp.value
        chaos = s_chaos.value
        speed = s_speed.value

        with torch.no_grad():
            # Inject context
            self.x = inject_context(self.x, ctx)

            # Simulate
            self.x = model(self.x) * temp

            # Add chaos
            if chaos > 0:
                noise = torch.randn_like(self.x) * chaos * 0.1
                self.x = self.x + noise

            metrics.update(self.x)

        self.step += int(speed)

    def run(self):
        self.running = True
        self.reset_state()

        while self.running:
            if not self.paused:
                self.update_step()

                if self.step % 1 == 0:
                    metrics_data = metrics.get_current()
                    ctx = [s_ctx1.value, s_ctx2.value, s_ctx3.value]
                    image_data = render_dashboard(
                        self.x, self.step, ctx,
                        s_temp.value, s_chaos.value, metrics_data
                    )
                    screen.value = image_data

            time.sleep(0.05)

controller = SimulationController()

# Event handlers
def on_pause(change):
    controller.paused = change['new']
    btn_pause.description = '▶️ RESUME' if controller.paused else '⏸️ PAUSE'

def on_reset(b):
    with output:
        controller.reset_state()
        print(f"✅ Reset - Step {controller.step}")

def on_snapshot(b):
    with output:
        filename = f"organism_snapshot_{time.strftime('%Y%m%d_%H%M%S')}.png"
        with open(filename, 'wb') as f:
            f.write(screen.value)
        print(f"📸 Saved: {filename}")

def on_stop(b):
    controller.running = False
    with output:
        print("🛑 Stopped")

btn_pause.observe(on_pause, names='value')
btn_reset.on_click(on_reset)
btn_snapshot.on_click(on_snapshot)
btn_stop.on_click(on_stop)

# Assemble UI
controls_row1 = widgets.HBox([s_ctx1, s_ctx2, s_ctx3])
controls_row2 = widgets.HBox([s_temp, s_chaos, s_speed])
buttons_row = widgets.HBox([btn_pause, btn_reset, btn_snapshot, btn_stop])

ui = widgets.VBox([
    widgets.HTML("<h2 style='text-align: center; color: cyan;'>🌌 DIGITAL ORGANISM OBSERVATORY 🌌</h2>"),
    controls_row1,
    controls_row2,
    buttons_row,
    screen,
    output
])

# Launch
print("\n" + "="*60)
print("🚀 DASHBOARD ONLINE")
print("="*60)
print("Features:")
print("  ✓ Real-time Neural Simulation")
print("  ✓ Entropy/Coherence/Complexity Tracking")
print("  ✓ Spectral Analysis")
print("  ✓ Interactive Parameter Control")
print("  ✓ Snapshot Capture")
print("="*60 + "\n")

display(ui)

thread = threading.Thread(target=controller.run, daemon=True)
thread.start()

# Version 2

In [ ]:
# CELL 1: TRUE INTELLIGENT DIGITAL ORGANISM
# Integrates: Memory, Curiosity, World Models, Self-Modification, Open-Ended Learning
# Train indefinitely - organism evolves genuine intelligence

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import shutil
import random
from collections import deque

# ==================== CONFIGURATION ====================
SEED = 42
CHANNELS = 32  # Increased for richer representations
GRID_SIZE = 64
MEMORY_SLOTS = 64  # External memory capacity
MEMORY_DIM = 16
BATCH_SIZE = 4
LEARNING_RATE = 1e-4
STEPS_PER_LIFE = 200  # Longer episodes for complex behavior
SAVE_INTERVAL = 20
CRYO_PATH = "intelligent_organism"

# ==================== INITIALIZATION ====================
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs(CRYO_PATH, exist_ok=True)

print(f"🧬 TRUE INTELLIGENT DIGITAL ORGANISM")
print(f"   Device: {device}")
print(f"   Architecture: Self-Modifying NCA + External Memory + World Model")
print(f"   Learning: Open-ended survival with intrinsic curiosity")
print(f"   Save: {CRYO_PATH}/")

# ==================== EXTERNAL MEMORY MODULE ====================
class SpatialMemory(nn.Module):
    """
    LEVEL 9 UPGRADE: Distributed Spatial Memory.
    Every cell on the grid (64x64) acts as an independent query head.
    This allows complex, non-uniform strategies across the organism's body.
    """
    def __init__(self, slots=MEMORY_SLOTS, dim=MEMORY_DIM):
        super().__init__()
        self.dim = dim
        self.slots = slots

        # Instinctive Memory (Ancestral Knowledge)
        self.instinct = nn.Parameter(torch.randn(1, slots, dim) * 0.02)

        # Spatial Projectors
        self.query_conv = nn.Conv2d(CHANNELS, dim, 1)
        self.key_conv   = nn.Linear(dim, dim)
        self.val_conv   = nn.Linear(dim, dim)

        # LEVEL 10 ADAPTABILITY: Differentiable Write Head
        self.write_query_head = nn.Conv2d(CHANNELS, dim, 1)
        self.write_gate = nn.Conv2d(CHANNELS, dim, 1) # What to write
        self.erase_gate = nn.Conv2d(CHANNELS, dim, 1) # What to forget

    def read(self, x, memory_state):
        """
        Retrieves information from the dynamic memory state
        memory_state: (Batch, Slots, Dim)
        """
        b, c, h, w = x.shape

        # 1. Generate Queries
        queries = self.query_conv(x).permute(0, 2, 3, 1).view(b, h*w, self.dim)

        # 2. Prepare Keys/Values
        mem_keys = self.key_conv(memory_state)
        mem_vals = self.val_conv(memory_state)

        # 3. Attention
        attn_logits = torch.bmm(queries, mem_keys.transpose(1, 2))
        attn_weights = F.softmax(attn_logits / (self.dim ** 0.5), dim=-1)

        # 4. Retrieve
        read_out = torch.bmm(attn_weights, mem_vals)

        # 5. Reshape
        return read_out.view(b, h, w, self.dim).permute(0, 3, 1, 2), attn_weights

    def write(self, x, memory_state):
        """
        Updates the dynamic memory state
        """
        b, c, h, w = x.shape

        # 1. Global Write Strategy
        write_queries = self.write_query_head(x).mean([2, 3])
        mem_keys = self.key_conv(memory_state)
        write_logits = torch.bmm(write_queries.unsqueeze(1), mem_keys.transpose(1, 2))
        write_weights = F.softmax(write_logits / (self.dim ** 0.5), dim=-1)

        # 2. Memory Content
        write_content = torch.tanh(self.write_gate(x).mean([2, 3])).unsqueeze(1)
        erase_mask = torch.sigmoid(self.erase_gate(x).mean([2, 3])).unsqueeze(1)

        # 3. Update
        e_gate = torch.bmm(write_weights.transpose(1, 2), erase_mask)
        w_gate = torch.bmm(write_weights.transpose(1, 2), write_content)

        return memory_state * (1 - e_gate) + w_gate

# ==================== WORLD MODEL ====================
# ==================== WORLD MODEL (FIXED) ====================
class WorldModel(nn.Module):
    """Predicts future states given actions (latent imagination)"""
    def __init__(self, state_dim=CHANNELS, action_dim=8):
        super().__init__()

        # 1. ENCODER: Compress Reality (64x64 -> 32x32)
        self.encoder = nn.Sequential(
            nn.Conv2d(state_dim, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 32, 3, stride=2, padding=1), # Downsample to 32x32
            nn.ReLU()
        )

        # 2. DYNAMICS: Process in Abstract Space
        # 32 channels * 32 height * 32 width = 32768 features
        self.dynamics = nn.GRUCell(32 * 32 * 32 + action_dim, 512)

        # 3. DECODER: Restore Imagination to Reality (32x32 -> 64x64)
        self.decoder_fc = nn.Sequential(
            nn.Linear(512, 32 * 32 * 32),
            nn.ReLU()
        )

        # FIX: Transposed Conv to upsample 32x32 back to 64x64
        self.decoder_upsample = nn.ConvTranspose2d(
            in_channels=32,
            out_channels=state_dim,
            kernel_size=3,
            stride=2,
            padding=1,
            output_padding=1
        )

        # LEVEL 5.5: Concept Predictor Head
        # Predicts what concepts will appear in the next state
        self.concept_head = nn.Sequential(
            nn.Linear(512, 32 * 32 * 8), # 8 Concepts
            nn.ReLU()
        )

        self.hidden = None

    def reset(self, batch_size):
        self.hidden = torch.zeros(batch_size, 512, device=device)

    def predict(self, state, action):
        """Predict next state given current state and action"""
        # A. Encode current state
        encoded = self.encoder(state)
        encoded_flat = encoded.reshape(state.size(0), -1) # Use reshape for safety

        # B. Combine with action and predict next hidden state
        inp = torch.cat([encoded_flat, action], dim=1)
        self.hidden = self.dynamics(inp, self.hidden)
        # STABILIZATION: Bound GRU hidden state
        self.hidden = torch.clamp(self.hidden, -10.0, 10.0)

        # C. Decode to predicted next state
        latent_spatial = self.decoder_fc(self.hidden)

        # Reshape back to spatial feature map (32x32)
        latent_spatial = latent_spatial.view(state.size(0), 32, 32, 32)

        # D. Upsample to full grid size (64x64)
        pred_state = self.decoder_upsample(latent_spatial)

        # STABILIZATION: Bound the prediction to prevent explosion
        return torch.tanh(pred_state) * 5.0

    def predict_concept(self):
        """Predicts the next concept distribution"""
        concepts = self.concept_head(self.hidden)
        return concepts.view(-1, 8, 32, 32) # Return raw logits

# ==================== SEMANTIC ENCODER (Step 1: Symbols) ====================
class SemanticEncoder(nn.Module):
    """
    Compresses raw spatial reality into discrete Concept Tokens.
    This is the bridge from "Pixels" to "Ideas".
    """
    def __init__(self, channels=CHANNELS, concepts=8):
        super().__init__()
        # 1x1 conv to extract pixel-wise concepts (e.g., Food, Danger, Wall)
        self.tokenizer = nn.Conv2d(channels, concepts, 1)
        self.concepts = concepts

    def forward(self, x):
        # Squeeze 32 channels down to 8 "Concept Channels"
        # We use Softmax to force the brain to CHOOSE a category per pixel
        logits = self.tokenizer(x)
        symbols = F.softmax(logits * 5.0, dim=1) # Temperature scaling for sharpness
        return symbols

# ==================== STRATEGIC CORTEX (Step 2: Slow Brain) ====================
class StrategicCortex(nn.Module):
    """
    The "Slow Brain" - Processes temporal sequences of concepts
    to form long-term strategies and goals.
    """
    def __init__(self, concept_dim=8, goal_dim=16):
        super().__init__()
        self.goal_dim = goal_dim
        # GRU to process concept history
        self.temporal_processor = nn.GRU(concept_dim, 64, batch_first=True)
        self.goal_generator = nn.Linear(64, goal_dim)
        self.hidden = None

    def reset(self, batch_size):
        self.hidden = torch.zeros(1, batch_size, 64, device=device)

    def forward(self, concept_summary):
        """
        concept_summary: (Batch, ConceptDim) - spatially averaged concepts
        """
        # Process as a single timestep in the recurrent brain
        out, self.hidden = self.temporal_processor(
            concept_summary.unsqueeze(1), self.hidden
        )
        # Generate goal vector
        goal = torch.tanh(self.goal_generator(out.squeeze(1)))
        return goal

# ==================== ARITHMETIC UNIT (Step 3: Symbolic Logic) ====================
class ArithmeticUnit(nn.Module):
    """
    The "Mental Scratchpad" - A non-spatial 1D memory for symbolic logic.
    Enables counting, comparisons, and abstract reasoning.
    """
    def __init__(self, ego_dim=64, concept_dim=8, scratchpad_dim=32):
        super().__init__()
        self.scratchpad_dim = scratchpad_dim
        # Logic processor: takes ego, concepts summary, and current scratchpad
        self.logic_gate = nn.Sequential(
            nn.Linear(ego_dim + concept_dim + scratchpad_dim, 64),
            nn.ReLU(),
            nn.Linear(64, scratchpad_dim * 2) # Output: [Erase, Write]
        )

    def forward(self, ego, concept_summary, scratchpad):
        """
        ego: (B, EgoDim)
        concept_summary: (B, ConceptDim)
        scratchpad: (B, ScratchpadDim)
        """
        combined = torch.cat([ego, concept_summary, scratchpad], dim=1)
        gates = self.logic_gate(combined)

        # Split into erase and write gates
        erase = torch.sigmoid(gates[:, :self.scratchpad_dim])
        write = torch.tanh(gates[:, self.scratchpad_dim:])

        # Update scratchpad: old * (1 - erase) + write
        new_scratchpad = scratchpad * (1 - erase) + write
        return new_scratchpad

# ==================== NEUROMODULATOR (Step 4: Meta-Cognition) ====================
class NeuroModulator(nn.Module):
    """
    The "Meta-Cognitive Controller" - The organism tunes its own brain.
    This is the final step to AGI: Self-Optimization.

    Outputs:
        - plasticity: How fast should I learn? (0=frozen, 1=maximum adaptation)
        - entropy: How random should my updates be? (0=precise, 1=exploratory)
        - imagination_trust: How much should I trust my World Model? (0=ignore, 1=fully trust)
    """
    def __init__(self, ego_dim=64, scratchpad_dim=32):
        super().__init__()
        # Input: Ego + Scratchpad + Prediction Error (scalar expanded)
        self.meta_brain = nn.Sequential(
            nn.Linear(ego_dim + scratchpad_dim + 1, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 3) # [plasticity, entropy, imagination_trust]
        )

    def forward(self, ego, scratchpad, prediction_error):
        """
        ego: (B, EgoDim)
        scratchpad: (B, ScratchpadDim)
        prediction_error: (B, 1) - How wrong was the World Model?
        """
        combined = torch.cat([ego, scratchpad, prediction_error], dim=1)
        raw_output = self.meta_brain(combined)

        # All outputs are gates (0 to 1)
        plasticity = torch.sigmoid(raw_output[:, 0:1])
        entropy = torch.sigmoid(raw_output[:, 1:2]) * 0.5 + 0.25 # Range: 0.25 to 0.75
        imagination_trust = torch.sigmoid(raw_output[:, 2:3])

        return plasticity, entropy, imagination_trust

# ==================== SELF-MODIFYING ORGANISM ====================
class IntelligentOrganism(nn.Module):
    def __init__(self, channels=CHANNELS, hidden=128, ego_dim=64):
        super().__init__()
        self.channels = channels
        self.ego_dim = ego_dim

        # LEVEL 10 ELEGANCE: Fixed Sobel Perception
        sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32)
        sobel_y = sobel_x.t()
        laplace = torch.tensor([[1, 2, 1], [2, -12, 2], [1, 2, 1]], dtype=torch.float32)
        perception_kernels = torch.stack([sobel_x, sobel_y, laplace])
        self.register_buffer('perception_kernels', perception_kernels.repeat(channels, 1, 1).unsqueeze(1))

        # LEVEL 10 AGENCY: The Glandular System & Ego Bottleneck
        # Proprioception: [Energy, Fatigue, Boredom, Pain]
        self.glandular_think = nn.Sequential(
            nn.Linear(4 + ego_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, ego_dim)
        )

        # LEVEL 5.5 GEN INTEL: Semantic Encoder
        self.semantic_encoder = SemanticEncoder(channels=channels, concepts=8)

        # LEVEL 8.0 GEN INTEL: Strategic Cortex (Slow Brain)
        self.strategic_cortex = StrategicCortex(concept_dim=8, goal_dim=16)

        # LEVEL 9.5 GEN INTEL: Arithmetic Unit (Mental Scratchpad)
        self.arithmetic_unit = ArithmeticUnit(ego_dim=ego_dim, concept_dim=8, scratchpad_dim=32)

        # LEVEL 10.0 GEN INTEL: NeuroModulator (Meta-Cognition)
        self.neuro_modulator = NeuroModulator(ego_dim=ego_dim, scratchpad_dim=32)

        # SENSORIUM: [Perceived(C*3)] + [Imagination(C)] + [Memory(M)] + [Ego(E)] + [Symbols(8)] + [Goal(16)] + [Scratchpad(32)]
        input_dim = channels * 3 + channels + MEMORY_DIM + ego_dim + 8 + 16 + 32

        self.think = nn.Sequential(
            nn.Conv2d(input_dim, hidden, 1),
            nn.GroupNorm(4, hidden),
            nn.LeakyReLU(0.1),
            nn.Conv2d(hidden, hidden, 3, padding=1, groups=hidden),
            nn.Conv2d(hidden, channels * 2 + 8, 1)
        )

        self.memory = SpatialMemory()
        self.register_buffer('null_imagination', torch.zeros(1, channels, GRID_SIZE, GRID_SIZE))

        # Persistent Internal State (The "Subjective Self")
        self.ego_state = None
        self.memory_state = None
        self.goal_state = None  # Strategic goal from Slow Brain
        self.scratchpad_state = None  # Mental Scratchpad for logic
        self.energy = 1.0
        self.boredom = 0.0

    def reset_ego(self, batch_size):
        """Resets the subjective self and local memory at start of life"""
        self.ego_state = torch.zeros(batch_size, self.ego_dim, device=device)
        self.memory_state = self.memory.instinct.expand(batch_size, -1, -1).clone()
        self.goal_state = torch.zeros(batch_size, 16, device=device)
        self.scratchpad_state = torch.zeros(batch_size, 32, device=device)
        self.prediction_error = torch.zeros(batch_size, 1, device=device)
        self.strategic_cortex.reset(batch_size)
        self.energy = torch.ones(batch_size, 1, device=device)
        self.boredom = torch.zeros(batch_size, 1, device=device)

    def forward(self, x, predicted_future=None, step=0, prediction_error=None):
        b, c, h, w = x.shape
        if predicted_future is None: predicted_future = torch.zeros_like(x)
        if self.ego_state is None: self.reset_ego(b)
        if prediction_error is not None: self.prediction_error = prediction_error

        # 0. META-COGNITION: Tune own brain before processing
        plasticity, entropy, imagination_trust = self.neuro_modulator(
            self.ego_state, self.scratchpad_state, self.prediction_error
        )

        # Gate the Imagination based on self-assessed trust
        # If organism doesn't trust its World Model, it ignores the dream
        imagination_gate = imagination_trust.view(b, 1, 1, 1)
        predicted_future = predicted_future * imagination_gate

        # 1. UPDATE GLANDULAR SYSTEM (Drives & Homeostasis)
        # Proprioception: awareness of internal needs
        proprioception = torch.cat([self.energy, self.boredom,
                                   torch.ones_like(self.energy) * (step/200.0), # Age
                                   torch.randn_like(self.energy) * 0.1], dim=1) # Noise

        # The Ego evolves based on its internal state
        self.ego_state = self.glandular_think(torch.cat([self.ego_state, proprioception], dim=1))
        # STABILIZATION: Bound the Ego
        self.ego_state = torch.clamp(self.ego_state, -5.0, 5.0)

        # 2. FIXED PERCEPTION
        with torch.no_grad():
            perceived = F.conv2d(x, self.perception_kernels, groups=c, padding=1)

        # 3. MEMORY RETRIEVAL (In-life experience)
        mem_read, _ = self.memory.read(x, self.memory_state)

        # 4. SEMANTIC ABSTRACTION (Symbolic Awakening)
        # What is this pixel? (Wall? Food? Empty?)
        symbols = self.semantic_encoder(x)

        # 5. STRATEGIC PLANNING (Slow Brain) - Every 10 steps
        # Summarize the current concept distribution
        concept_summary = symbols.mean([2, 3]) # (B, 8)
        if step % 10 == 0:
            self.goal_state = self.strategic_cortex(concept_summary)

        # 6. SYMBOLIC LOGIC (Mental Scratchpad)
        # The organism performs abstract reasoning on its concepts
        self.scratchpad_state = self.arithmetic_unit(self.ego_state, concept_summary, self.scratchpad_state)
        # STABILIZATION: Bound the Scratchpad
        self.scratchpad_state = torch.clamp(self.scratchpad_state, -2.0, 2.0)

        # 7. SENSORIUM FUSION (Now with Scratchpad Injection)
        ego_spatial = self.ego_state.view(b, self.ego_dim, 1, 1).expand(-1, -1, h, w)
        goal_spatial = self.goal_state.view(b, 16, 1, 1).expand(-1, -1, h, w)
        scratchpad_spatial = self.scratchpad_state.view(b, 32, 1, 1).expand(-1, -1, h, w)
        sensorium = torch.cat([perceived, predicted_future, mem_read, ego_spatial, symbols, goal_spatial, scratchpad_spatial], dim=1)

        # 8. NEURAL PROCESSING
        raw_output = self.think(sensorium)

        # Split outputs
        gate          = torch.sigmoid(raw_output[:, :self.channels])
        update_vector = torch.tanh(raw_output[:, self.channels:self.channels*2])
        actions       = torch.tanh(raw_output[:, self.channels*2:])

        # 9. UPDATE BODY & INTERNAL DRIVES
        # META-COGNITION: Use self-determined entropy for stochastic updates
        entropy_threshold = entropy.view(b, 1, 1, 1).expand(-1, 1, h, w)
        stochastic_mask = (torch.rand(b, 1, h, w, device=x.device) > entropy_threshold).float()
        x_new = x + stochastic_mask * gate * update_vector

        # STABILIZATION: Apply physical bounds to the cellular state
        x_new = torch.clamp(x_new, -5.0, 5.0)

        # 7. COMMIT TO MEMORY (Real-time learning)
        # The organism writes its current state and "thoughts" to memory
        self.memory_state = self.memory.write(x_new, self.memory_state)
        # STABILIZATION: Keep memory bounded
        self.memory_state = torch.clamp(self.memory_state, -2.0, 2.0)

        # Metabolize: Update energy based on movement magnitude
        movement = actions.abs().mean([1, 2, 3]).view(b, 1)
        self.energy = (self.energy - movement * 0.01).clamp(0, 1)
        self.boredom = (self.boredom + 0.005).clamp(0, 1) # Boredom increases over time

        # STABILIZATION: Bound Slow Brain state
        if self.strategic_cortex.hidden is not None:
            self.strategic_cortex.hidden = torch.clamp(self.strategic_cortex.hidden, -2.0, 2.0)

        return x_new, actions

# ==================== OPEN-ENDED ENVIRONMENT ====================
def make_dynamic_environment(batch_size, step):
    """Creates evolving environmental challenges"""
    env = torch.zeros(batch_size, CHANNELS, GRID_SIZE, GRID_SIZE, device=device)

    # Dynamic obstacles and resources
    for i in range(batch_size):
        # Moving resources
        resource_x = int(32 + 20 * np.sin(step * 0.05 + i))
        resource_y = int(32 + 20 * np.cos(step * 0.05 + i))
        env[i, 0:4, resource_y-3:resource_y+3, resource_x-3:resource_x+3] = 1.0

        # Random obstacles
        if random.random() < 0.3:
            obs_x = random.randint(10, 54)
            obs_y = random.randint(10, 54)
            env[i, 4:8, obs_y-2:obs_y+2, obs_x-2:obs_x+2] = -1.0

    return env

def compute_survival_fitness(organism_state, environment, actions=None):
    """Measures organism's survival capability with metabolic constraints"""
    # Energy: How much "alive" mass does it maintain?
    energy = organism_state[:, :8].abs().sum([1, 2, 3])

    # Coherence: Does it maintain structure?
    variance = organism_state[:, :8].var([1, 2, 3])
    coherence = 1.0 / (1.0 + variance)

    # Resource collection: Does it move toward resources?
    resource_locations = (environment[:, 0] > 0.5).float()
    organism_density = organism_state[:, 0].abs()
    overlap = (resource_locations * organism_density).sum([1, 2])

    # Avoid obstacles
    obstacle_locations = (environment[:, 4] < -0.5).float()
    obstacle_hit = (obstacle_locations * organism_density).sum([1, 2])

    # LEVEL 10: Metabolic Penalty (Efficiency matters for AGI)
    metabolic_cost = 0
    if actions is not None:
        metabolic_cost = actions.abs().mean([1, 2, 3]) * 2.0

    # STABILIZATION: Scale down energy reward to prevent explosion
    # Use log-scaling or a small multiplier
    normalized_energy = torch.log1p(energy) * 2.0

    fitness = normalized_energy + coherence * 10.0 + overlap * 5.0 - obstacle_hit * 3.0 - metabolic_cost
    return fitness

# ==================== PERSISTENCE ====================
class CryoStorage:
    def freeze(self, organism, world_model, optimizer_org, optimizer_wm, epoch, metrics):
        filepath = os.path.join(CRYO_PATH, f"organism_epoch_{epoch}.pth")

        capsule = {
            'organism': organism.state_dict(),
            'world_model': world_model.state_dict(),
            'optimizer_org': optimizer_org.state_dict(),
            'optimizer_wm': optimizer_wm.state_dict(),
            'memory_instinct': organism.memory.instinct.cpu(),
            'epoch': epoch,
            'metrics': metrics,
            'rng_state': torch.get_rng_state()
        }

        torch.save(capsule, filepath)
        shutil.copy(filepath, os.path.join(CRYO_PATH, "organism_latest.pth"))
        shutil.make_archive("intelligent_organism_backup", 'zip', CRYO_PATH)

        print(f"💾 Saved Epoch {epoch} | Fitness: {metrics['fitness'][-1]:.2f} | Curiosity: {metrics['curiosity'][-1]:.4f}")
        return filepath

    def resurrect(self, organism, world_model, optimizer_org, optimizer_wm):
        filepath = os.path.join(CRYO_PATH, "organism_latest.pth")

        if not os.path.exists(filepath):
            print("🌱 Genesis: Creating new intelligent life")
            return 0, {'fitness': [], 'curiosity': [], 'entropy': []}

        print(f"⚡ Resurrecting from {filepath}...")
        capsule = torch.load(filepath, map_location=device)

        organism.load_state_dict(capsule['organism'])
        world_model.load_state_dict(capsule['world_model'])
        optimizer_org.load_state_dict(capsule['optimizer_org'])
        optimizer_wm.load_state_dict(capsule['optimizer_wm'])
        organism.memory.instinct.data = capsule['memory_instinct'].to(device)
        torch.set_rng_state(capsule['rng_state'].cpu())

        print(f"✅ Restored Epoch {capsule['epoch']} | Intelligence preserved")
        return capsule['epoch'] + 1, capsule['metrics']

# ==================== TRAINING ENGINE ====================
organism = IntelligentOrganism(CHANNELS).to(device)
world_model = WorldModel().to(device)
optimizer_org = torch.optim.Adam(organism.parameters(), lr=LEARNING_RATE)
optimizer_wm = torch.optim.Adam(world_model.parameters(), lr=LEARNING_RATE * 2)
cryo = CryoStorage()

start_epoch, metrics = cryo.resurrect(organism, world_model, optimizer_org, optimizer_wm)

print(f"\n{'='*70}")
print(f"🚀 TRAINING INTELLIGENT ORGANISM")
print(f"   Epoch: {start_epoch} | Memory Slots: {MEMORY_SLOTS}")
print(f"   Capabilities: Curiosity, Memory, World Modeling, Self-Modification")
print(f"   Goal: Open-ended survival & intelligence emergence")
print(f"{'='*70}\n")

# ==================== LEVEL 9 TRAINING LOOP ====================
# Configuration for Stability
TBPTT_STEPS = 20  # How many steps we look back for gradients
burn_in_steps = 0

print("🚀 INITIATING LEVEL 9 INTELLIGENCE ENGINE...")

try:
    epoch = start_epoch
    while True:
        # 1. Reset Environment (New Life)
        x = torch.randn(BATCH_SIZE, CHANNELS, GRID_SIZE, GRID_SIZE, device=device) * 0.1
        world_model.reset(BATCH_SIZE)
        organism.reset_ego(BATCH_SIZE)

        # 2. Long Life Simulation
        # We don't recreate the optimizer; we detach the state.
        for life_step in range(0, STEPS_PER_LIFE, TBPTT_STEPS):

            optimizer_org.zero_grad()
            optimizer_wm.zero_grad()

            loss_chunk = 0

            # Short-term rollouts for gradient calculation
            for t in range(TBPTT_STEPS):
                total_step = life_step + t

                # Dynamic Environment (Predator/Prey logic would go here)
                env = make_dynamic_environment(BATCH_SIZE, total_step + epoch * 1000)

                # A. DREAMING (World Model Prediction)
                # "If I stay here, what happens?"
                with torch.no_grad():
                    imagination = world_model.predict(x[:, :32], torch.zeros(BATCH_SIZE, 8, device=device))

                # B. CALCULATE PREDICTION ERROR (For Meta-Cognition)
                # How wrong was the World Model's last prediction?
                # Use the organism's internal state to track this
                if t > 0:
                    prediction_error = F.mse_loss(imagination, x[:, :32], reduction='none').mean([1,2,3]).view(BATCH_SIZE, 1)
                else:
                    prediction_error = torch.zeros(BATCH_SIZE, 1, device=device)

                # STABILIZATION: Ensure prediction_error is never NaN
                prediction_error = torch.nan_to_num(prediction_error, nan=0.0)

                # C. ACTING (With Meta-Cognitive Feedback)
                x_prev = x
                x_next, actions = organism(x + env * 0.05, imagination, total_step, prediction_error)

                # D. WORLD MODEL TRAINING (Self-Supervised)
                wm_pred = world_model.predict(x[:, :32].detach(), actions.mean([2,3]).detach())
                wm_loss = F.mse_loss(wm_pred, x_next[:, :32].detach())

                # SEMANTIC LOSS: Predict the concepts of the next state
                # Downsample target concepts to match predictor (64x64 -> 32x32)
                next_concepts = organism.semantic_encoder(x_next)
                next_concepts_small = F.interpolate(next_concepts, size=(32, 32), mode='bilinear')
                pred_concepts = world_model.predict_concept()

                # Cross Entropy (Softmax is implicit in CrossEntropyLoss, but we have soft targets)
                # We use MSE for soft-target probability matching
                concept_loss = F.mse_loss(torch.softmax(pred_concepts, dim=1), next_concepts_small.detach())

                # D. FITNESS CALCULATION
                fitness = compute_survival_fitness(x_next, env, actions)

                # Loss accumulation
                # Maximize Fitness - Minimize WM Error - Minimize Concept Error
                loss_chunk += -fitness.mean() + wm_loss + concept_loss * 0.1

                # Move state forward
                x = x_next

            # E. BACKPROPAGATION (The Learning)
            loss_chunk.backward()

            # Clip gradients to prevent "Exploding Brain" syndrome
            nn.utils.clip_grad_norm_(organism.parameters(), 0.5)
            nn.utils.clip_grad_norm_(world_model.parameters(), 0.5)

            optimizer_org.step()
            optimizer_wm.step()

            # F. DETACH STATE (The Cheat-Prevention for Physics)
            # We stop gradients from flowing back to the "big bang" of the episode.
            x = x.detach()
            world_model.hidden = world_model.hidden.detach()
            organism.ego_state = organism.ego_state.detach()
            organism.memory_state = organism.memory_state.detach()
            organism.goal_state = organism.goal_state.detach()
            organism.scratchpad_state = organism.scratchpad_state.detach()
            organism.strategic_cortex.hidden = organism.strategic_cortex.hidden.detach()
            organism.energy = organism.energy.detach()
            organism.boredom = organism.boredom.detach()

        epoch += 1

        # Update metrics to ensure saving works
        metrics['fitness'].append(-loss_chunk.item()/TBPTT_STEPS)
        metrics['curiosity'].append(wm_loss.item() if 'wm_loss' in locals() else 0.0)

        if epoch % 10 == 0:
            print(f"Epoch {epoch} | Survival Fitness: {-loss_chunk.item()/TBPTT_STEPS:.4f}")

except KeyboardInterrupt:
    print("\n🛑 Manual Interruption: Stabilizing quantum state...")
    # Do nothing here. We let 'finally' handle the heavy lifting.

except Exception as e:
    # 0% Cheat: Catch actual code errors too, not just manual stops
    print(f"\n⚠️ CRITICAL FAILURE: {str(e)}")
    # We still fall through to 'finally' to save what we have

finally:
    print("\n💾 Emergency Cryo-Freeze Sequence Initiated...")

    # SAFETY CHECK: Only save if the organism actually exists and started living
    if 'organism' in locals() and 'metrics' in locals():
        # Prevent "IndexError" if you stop it before the first log
        if len(metrics['fitness']) > 0:
            cryo.freeze(organism, world_model, optimizer_org, optimizer_wm, epoch, metrics)
            print("✅ Complete. Download: intelligent_organism_backup.zip")
            print(f"   Intelligence evolved over {epoch} epochs")
            print(f"   Final Fitness: {metrics['fitness'][-1]:.2f}")
            print(f"   Curiosity: {metrics['curiosity'][-1]:.4f}")
        else:
            print("⚠️ Simulation aborted before first epoch. No data to save.")
    else:
        print("❌ System initialization failed. No organism found.")

🧬 TRUE INTELLIGENT DIGITAL ORGANISM
   Device: cuda
   Architecture: Self-Modifying NCA + External Memory + World Model
   Learning: Open-ended survival with intrinsic curiosity
   Save: intelligent_organism/
🌱 Genesis: Creating new intelligent life

🚀 TRAINING INTELLIGENT ORGANISM
   Epoch: 0 | Memory Slots: 64
   Capabilities: Curiosity, Memory, World Modeling, Self-Modification
   Goal: Open-ended survival & intelligence emergence

🚀 INITIATING LEVEL 9 INTELLIGENCE ENGINE...
Epoch 10 | Survival Fitness: 847.9250
Epoch 20 | Survival Fitness: 829.2590
Epoch 30 | Survival Fitness: 870.0968
Epoch 40 | Survival Fitness: 849.7498
Epoch 50 | Survival Fitness: 853.1060
Epoch 60 | Survival Fitness: 832.0816
Epoch 70 | Survival Fitness: 864.9053

🛑 Manual Interruption: Stabilizing quantum state...

💾 Emergency Cryo-Freeze Sequence Initiated...
💾 Saved Epoch 71 | Fitness: 868.20 | Curiosity: 0.1658
✅ Complete. Download: intelligent_organism_backup.zip
   Intelligence evolved over 71 epochs
   

In [ ]:
# CELL 2: INTELLIGENCE VISUALIZATION DASHBOARD
# Real-time monitoring of organism's evolving intelligence

import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import animation
import ipywidgets as widgets
from IPython.display import display, Image
from collections import deque
import io
import time
import threading

# ==================== SETUP ====================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CRYO_PATH = "intelligent_organism"

print("🔬 Intelligence Observatory Online...")

# ==================== ARCHITECTURE RECONSTRUCTION ====================
# ==================== ARCHITECTURE RECONSTRUCTION ====================
# Global Parameters (Synced with Cell1.py)
CHANNELS = 32
GRID_SIZE = 64
MEMORY_SLOTS = 64
MEMORY_DIM = 16

class SpatialMemory(nn.Module):
    def __init__(self, slots=MEMORY_SLOTS, dim=MEMORY_DIM):
        super().__init__()
        self.dim = dim
        self.slots = slots
        self.instinct = nn.Parameter(torch.randn(1, slots, dim) * 0.02)
        self.query_conv = nn.Conv2d(CHANNELS, dim, 1)
        self.key_conv   = nn.Linear(dim, dim)
        self.val_conv   = nn.Linear(dim, dim)
        self.write_query_head = nn.Conv2d(CHANNELS, dim, 1)
        self.write_gate = nn.Conv2d(CHANNELS, dim, 1)
        self.erase_gate = nn.Conv2d(CHANNELS, dim, 1)

    def read(self, x, memory_state):
        b, c, h, w = x.shape
        queries = self.query_conv(x).permute(0, 2, 3, 1).view(b, h*w, self.dim)
        mem_keys = self.key_conv(memory_state)
        mem_vals = self.val_conv(memory_state)
        attn_logits = torch.bmm(queries, mem_keys.transpose(1, 2))
        attn_weights = torch.nn.functional.softmax(attn_logits / (self.dim ** 0.5), dim=-1)
        read_out = torch.bmm(attn_weights, mem_vals)
        return read_out.view(b, h, w, self.dim).permute(0, 3, 1, 2), attn_weights

    def write(self, x, memory_state):
        b, c, h, w = x.shape
        write_queries = self.write_query_head(x).mean([2, 3])
        mem_keys = self.key_conv(memory_state)
        write_logits = torch.bmm(write_queries.unsqueeze(1), mem_keys.transpose(1, 2))
        write_weights = torch.nn.functional.softmax(write_logits / (self.dim ** 0.5), dim=-1)
        write_content = torch.tanh(self.write_gate(x).mean([2, 3])).unsqueeze(1)
        erase_mask = torch.sigmoid(self.erase_gate(x).mean([2, 3])).unsqueeze(1)
        e_gate = torch.bmm(write_weights.transpose(1, 2), erase_mask)
        w_gate = torch.bmm(write_weights.transpose(1, 2), write_content)
        return memory_state * (1 - e_gate) + w_gate

class WorldModel(nn.Module):
    def __init__(self, state_dim=CHANNELS, action_dim=8):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(state_dim, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 32, 3, stride=2, padding=1),
            nn.ReLU()
        )
        self.dynamics = nn.GRUCell(32 * 32 * 32 + action_dim, 512)
        self.decoder_fc = nn.Sequential(nn.Linear(512, 32 * 32 * 32), nn.ReLU())
        self.decoder_upsample = nn.ConvTranspose2d(32, state_dim, 3, stride=2, padding=1, output_padding=1)
        self.concept_head = nn.Sequential(nn.Linear(512, 32 * 32 * 8), nn.ReLU())
        self.hidden = None

    def reset(self, batch_size):
        self.hidden = torch.zeros(batch_size, 512, device=device)

    def predict(self, state, action):
        encoded = self.encoder(state)
        encoded_flat = encoded.view(state.size(0), -1)
        inp = torch.cat([encoded_flat, action], dim=1)
        self.hidden = self.dynamics(inp, self.hidden)
        # STABILIZATION
        self.hidden = torch.clamp(self.hidden, -10.0, 10.0)
        latent_spatial = self.decoder_fc(self.hidden).view(state.size(0), 32, 32, 32)
        pred_state = self.decoder_upsample(latent_spatial)
        return torch.tanh(pred_state) * 5.0

class SemanticEncoder(nn.Module):
    def __init__(self, channels=CHANNELS, concepts=8):
        super().__init__()
        self.tokenizer = nn.Conv2d(channels, concepts, 1)

    def forward(self, x):
        logits = self.tokenizer(x)
        return torch.nn.functional.softmax(logits * 5.0, dim=1)

class StrategicCortex(nn.Module):
    def __init__(self, concept_dim=8, goal_dim=16):
        super().__init__()
        self.temporal_processor = nn.GRU(concept_dim, 64, batch_first=True)
        self.goal_generator = nn.Linear(64, goal_dim)
        self.hidden = None

    def reset(self, batch_size):
        self.hidden = torch.zeros(1, batch_size, 64, device=device)

    def forward(self, concept_summary):
        out, self.hidden = self.temporal_processor(concept_summary.unsqueeze(1), self.hidden)
        return torch.tanh(self.goal_generator(out.squeeze(1)))

class ArithmeticUnit(nn.Module):
    def __init__(self, ego_dim=64, concept_dim=8, scratchpad_dim=32):
        super().__init__()
        self.scratchpad_dim = scratchpad_dim
        self.logic_gate = nn.Sequential(
            nn.Linear(ego_dim + concept_dim + scratchpad_dim, 64),
            nn.ReLU(),
            nn.Linear(64, scratchpad_dim * 2)
        )

    def forward(self, ego, concept_summary, scratchpad):
        gates = self.logic_gate(torch.cat([ego, concept_summary, scratchpad], dim=1))
        erase = torch.sigmoid(gates[:, :self.scratchpad_dim])
        write = torch.tanh(gates[:, self.scratchpad_dim:])
        return scratchpad * (1 - erase) + write

class NeuroModulator(nn.Module):
    def __init__(self, ego_dim=64, scratchpad_dim=32):
        super().__init__()
        self.meta_brain = nn.Sequential(
            nn.Linear(ego_dim + scratchpad_dim + 1, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 3)
        )

    def forward(self, ego, scratchpad, prediction_error):
        raw = self.meta_brain(torch.cat([ego, scratchpad, prediction_error], dim=1))
        plasticity = torch.sigmoid(raw[:, 0:1])
        entropy = torch.sigmoid(raw[:, 1:2]) * 0.5 + 0.25
        imagination_trust = torch.sigmoid(raw[:, 2:3])
        return plasticity, entropy, imagination_trust

class IntelligentOrganism(nn.Module):
    def __init__(self, channels=CHANNELS, hidden=128, ego_dim=64):
        super().__init__()
        self.channels, self.ego_dim = channels, ego_dim
        sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32)
        sobel_y = sobel_x.t()
        laplace = torch.tensor([[1, 2, 1], [2, -12, 2], [1, 2, 1]], dtype=torch.float32)
        perception_kernels = torch.stack([sobel_x, sobel_y, laplace])
        self.register_buffer('perception_kernels', perception_kernels.repeat(channels, 1, 1).unsqueeze(1))
        self.glandular_think = nn.Sequential(nn.Linear(4 + ego_dim, hidden), nn.ReLU(), nn.Linear(hidden, ego_dim))
        self.semantic_encoder = SemanticEncoder(channels, 8)
        self.strategic_cortex = StrategicCortex(8, 16)
        self.arithmetic_unit = ArithmeticUnit(ego_dim, 8, 32)
        self.neuro_modulator = NeuroModulator(ego_dim, 32)
        input_dim = channels * 3 + channels + MEMORY_DIM + ego_dim + 8 + 16 + 32
        self.think = nn.Sequential(
            nn.Conv2d(input_dim, hidden, 1),
            nn.GroupNorm(4, hidden),
            nn.LeakyReLU(0.1),
            nn.Conv2d(hidden, hidden, 3, padding=1, groups=hidden),
            nn.Conv2d(hidden, channels * 2 + 8, 1)
        )
        self.memory = SpatialMemory()
        self.register_buffer('null_imagination', torch.zeros(1, channels, GRID_SIZE, GRID_SIZE))
        self.ego_state = None
        self.memory_state = None
        self.goal_state = None
        self.scratchpad_state = None
        self.energy = None
        self.boredom = None
        self.prediction_error = None
        self.semantic_concepts = None # For visualization

    def reset_ego(self, b):
        self.ego_state = torch.zeros(b, self.ego_dim, device=device)
        self.memory_state = self.memory.instinct.expand(b, -1, -1).clone()
        self.goal_state = torch.zeros(b, 16, device=device)
        self.scratchpad_state = torch.zeros(b, 32, device=device)
        self.prediction_error = torch.zeros(b, 1, device=device)
        self.strategic_cortex.reset(b)
        self.energy = torch.ones(b, 1, device=device)
        self.boredom = torch.zeros(b, 1, device=device)

    def forward(self, x, predicted_future=None, step=0, prediction_error=None):
        b, c, h, w = x.shape
        if self.ego_state is None: self.reset_ego(b)
        if predicted_future is None: predicted_future = torch.zeros_like(x)
        if prediction_error is not None: self.prediction_error = prediction_error

        plasticity, entropy, imagination_trust = self.neuro_modulator(self.ego_state, self.scratchpad_state, self.prediction_error)
        predicted_future = predicted_future * imagination_trust.view(b, 1, 1, 1)

        proprioception = torch.cat([self.energy, self.boredom, torch.ones_like(self.energy)*(step/200.0), torch.randn_like(self.energy)*0.1], dim=1)
        self.ego_state = torch.clamp(self.glandular_think(torch.cat([self.ego_state, proprioception], dim=1)), -5.0, 5.0)

        with torch.no_grad(): perceived = torch.nn.functional.conv2d(x, self.perception_kernels, groups=c, padding=1)
        mem_read, attn = self.memory.read(x, self.memory_state)
        symbols = self.semantic_encoder(x)
        concept_summary = symbols.mean([2, 3])
        self.semantic_concepts = concept_summary # Save for observatory
        if step % 10 == 0: self.goal_state = self.strategic_cortex(concept_summary)
        self.scratchpad_state = torch.clamp(self.arithmetic_unit(self.ego_state, concept_summary, self.scratchpad_state), -2.0, 2.0)

        ego_spatial = self.ego_state.view(b, self.ego_dim, 1, 1).expand(-1, -1, h, w)
        goal_spatial = self.goal_state.view(b, 16, 1, 1).expand(-1, -1, h, w)
        scratchpad_spatial = self.scratchpad_state.view(b, 32, 1, 1).expand(-1, -1, h, w)
        sensorium = torch.cat([perceived, predicted_future, mem_read, ego_spatial, symbols, goal_spatial, scratchpad_spatial], dim=1)

        raw = self.think(sensorium)
        gate, update_vector, actions = torch.sigmoid(raw[:, :c]), torch.tanh(raw[:, c:c*2]), torch.tanh(raw[:, c*2:])

        stochastic_mask = (torch.rand(b, 1, h, w, device=x.device) > entropy.view(b,1,1,1)).float()
        x_new = torch.clamp(x + stochastic_mask * gate * update_vector, -5.0, 5.0)
        self.memory_state = torch.clamp(self.memory.write(x_new, self.memory_state), -2.0, 2.0)

        self.energy = (self.energy - actions.abs().mean([1,2,3]).view(b,1)*0.01).clamp(0, 1)
        self.boredom = (self.boredom + 0.005).clamp(0, 1)
        if self.strategic_cortex.hidden is not None: self.strategic_cortex.hidden = torch.clamp(self.strategic_cortex.hidden, -2.0, 2.0)

        return x_new, actions, (plasticity, entropy, imagination_trust)

# Load organism
model = IntelligentOrganism(32, 128).to(device)
world_model = WorldModel().to(device)

try:
    checkpoint = torch.load(f"{CRYO_PATH}/organism_latest.pth", map_location=device)
    model.load_state_dict(checkpoint['organism'])
    world_model.load_state_dict(checkpoint['world_model'])
    model.memory.instinct.data = checkpoint['memory_instinct'].to(device)
    model.eval()
    world_model.eval()
    print(f"✅ Loaded 10/10 Nano-AGI from Epoch {checkpoint['epoch']}")
except Exception as e:
    print(f"⚠️ No trained organism found or error: {str(e)}. Using random initialization.")

# ==================== METRICS ENGINE ====================
class IntelligenceMetrics:
    def __init__(self, history=300):
        self.fitness = deque(maxlen=history)
        self.curiosity = deque(maxlen=history)
        self.entropy = deque(maxlen=history)
        self.memory_activity = deque(maxlen=history)
        self.coherence = deque(maxlen=history)
        self.prediction_error = deque(maxlen=history)

    def calculate_metrics(self, x, mem_activity, curiosity, pred_error, entropy_val=0.5):
        # Physical Entropy
        p = x.abs()
        p = p / (p.sum() + 1e-8)
        phys_entropy = -(p * torch.log(p + 1e-8)).sum().item()

        # Coherence (Structure)
        coherence = 1.0 / (1.0 + x[:, :8].var().item())

        # Fitness
        energy = x[:, :8].abs().sum().item()
        fitness = energy * 0.1 + coherence * 10.0

        self.fitness.append(fitness)
        self.curiosity.append(curiosity)
        self.entropy.append(entropy_val)
        self.memory_activity.append(mem_activity)
        self.coherence.append(coherence)
        self.prediction_error.append(pred_error)

    def get_current(self):
        return {
            'fitness': self.fitness[-1] if self.fitness else 0,
            'curiosity': self.curiosity[-1] if self.curiosity else 0,
            'entropy': self.entropy[-1] if self.entropy else 0,
            'memory_activity': self.memory_activity[-1] if self.memory_activity else 0,
            'coherence': self.coherence[-1] if self.coherence else 0,
            'prediction_error': self.prediction_error[-1] if self.prediction_error else 0
        }

metrics = IntelligenceMetrics()

# ==================== ENVIRONMENT ====================
def make_env(step):
    env = torch.zeros(1, 32, 64, 64, device=device)
    resource_x = int(32 + 20 * np.sin(step * 0.05))
    resource_y = int(32 + 20 * np.cos(step * 0.05))
    env[0, 0:4, resource_y-3:resource_y+3, resource_x-3:resource_x+3] = 1.0
    if random.random() < 0.3:
        obs_x = random.randint(10, 54)
        obs_y = random.randint(10, 54)
        env[0, 4:8, obs_y-2:obs_y+2, obs_x-2:obs_x+2] = -1.0
    return env

def get_intelligence_level(fitness, curiosity, coherence):
    score = fitness * 0.3 + curiosity * 50 + coherence * 20
    if score < 5: return "Dormant", "#444444"
    elif score < 15: return "Awakening", "#666699"
    elif score < 30: return "Curious", "#4488ff"
    elif score < 50: return "Learning", "#00cc88"
    elif score < 80: return "Intelligent", "#ffaa00"
    else: return "TRANSCENDENT", "#ff00ff"

# ==================== DASHBOARD RENDERER ====================
def render_dashboard(x, step, env, mem_attn, metrics_data):
    img_body = x[0, :3].permute(1, 2, 0).detach().cpu().clamp(0, 1).numpy()
    img_brain = x[0, 8].detach().cpu().numpy()
    img_memory = x[0, 16:19].permute(1, 2, 0).detach().cpu().clamp(0, 1).numpy()
    img_env = env[0, :3].permute(1, 2, 0).detach().cpu().clamp(0, 1).numpy()
    mem_heatmap = mem_attn[0].detach().cpu().mean(dim=0).numpy() # Global attention across all pixels

    plt.style.use('dark_background')
    fig = plt.figure(figsize=(22, 12), facecolor='#0a0a0a')
    gs = GridSpec(4, 6, figure=fig, hspace=0.35, wspace=0.35)

    # Main body
    ax_body = fig.add_subplot(gs[:2, :2])
    ax_body.imshow(img_body)
    intel_level, intel_color = get_intelligence_level(
        metrics_data['fitness'],
        metrics_data['curiosity'],
        metrics_data['coherence']
    )
    ax_body.set_title(f"🧬 ORGANISM BODY\nIntelligence: {intel_level}",
                     color=intel_color, fontsize=16, fontweight='bold', pad=15)
    ax_body.axis('off')

    # Brain activity
    ax_brain = fig.add_subplot(gs[0, 2])
    im = ax_brain.imshow(img_brain, cmap='magma')
    ax_brain.set_title("🧠 NEURAL ACTIVITY", color='orange', fontsize=11)
    ax_brain.axis('off')
    plt.colorbar(im, ax=ax_brain, fraction=0.046)

    # Memory state
    ax_mem = fig.add_subplot(gs[0, 3])
    ax_mem.imshow(img_memory)
    ax_mem.set_title("💾 MEMORY PATTERNS", color='lime', fontsize=11)
    ax_mem.axis('off')

    # Environment
    ax_env = fig.add_subplot(gs[1, 2])
    ax_env.imshow(img_env)
    ax_env.set_title("🌍 ENVIRONMENT", color='cyan', fontsize=11)
    ax_env.axis('off')

    # Memory attention
    ax_attn = fig.add_subplot(gs[1, 3])
    ax_attn.bar(range(len(mem_heatmap)), mem_heatmap, color='violet', alpha=0.7)
    ax_attn.set_title("🎯 MEMORY ATTENTION", color='violet', fontsize=11)
    ax_attn.set_ylim(0, max(mem_heatmap) * 1.2 if max(mem_heatmap) > 0 else 1)
    ax_attn.grid(True, alpha=0.2)

    # Fitness timeline
    ax_fit = fig.add_subplot(gs[2, :2])
    if len(metrics.fitness) > 1:
        ax_fit.plot(list(metrics.fitness), color='lime', linewidth=2, alpha=0.8)
        ax_fit.fill_between(range(len(metrics.fitness)), list(metrics.fitness),
                           alpha=0.3, color='lime')
    ax_fit.set_title("📊 FITNESS (Survival)", color='lime', fontsize=11)
    ax_fit.grid(True, alpha=0.2)
    ax_fit.set_ylabel('Fitness', color='white')

    # Trust timeline
    ax_cur = fig.add_subplot(gs[2, 2:4])
    if len(metrics.curiosity) > 1:
        ax_cur.plot(list(metrics.curiosity), color='magenta', linewidth=2, alpha=0.8)
    ax_cur.set_title("🔮 IMAGINATION TRUST", color='magenta', fontsize=11)
    ax_cur.grid(True, alpha=0.2)
    ax_cur.set_ylabel('Trust', color='white')

    # Coherence timeline
    ax_coh = fig.add_subplot(gs[3, :2])
    if len(metrics.coherence) > 1:
        ax_coh.plot(list(metrics.coherence), color='cyan', linewidth=2, alpha=0.8)
    ax_coh.set_title("🎯 COHERENCE (Structure)", color='cyan', fontsize=11)
    ax_coh.grid(True, alpha=0.2)
    ax_coh.set_ylabel('Coherence', color='white')
    ax_coh.set_xlabel('Time', color='white')

    # Entropy timeline
    ax_ent = fig.add_subplot(gs[3, 2:4])
    if len(metrics.entropy) > 1:
        ax_ent.plot(list(metrics.entropy), color='yellow', linewidth=2, alpha=0.8)
    ax_ent.set_title("⚡ ENTROPY (Complexity)", color='yellow', fontsize=11)
    ax_ent.grid(True, alpha=0.2)
    ax_ent.set_ylabel('Entropy', color='white')
    ax_ent.set_xlabel('Time', color='white')

    # Metrics panel
    ax_metrics = fig.add_subplot(gs[:2, 4:])
    ax_metrics.axis('off')

    metrics_text = f"""
╔═══════════════════════════════════╗
║   🎯 INTELLIGENCE METRICS         ║
╚═══════════════════════════════════╝

⏱️  TIMESTEP: {step:,}

📊 FITNESS:          {metrics_data['fitness']:.2f}
🔮 IMAGINATION TRUST:{metrics_data['curiosity']:.4f}
⚡ ENTROPY:          {metrics_data['entropy']:.3f}
💾 MEMORY ACTIVITY:  {metrics_data['memory_activity']:.1f}
🎯 COHERENCE:        {metrics_data['coherence']:.3f}
🧠 PREDICTION ERR:   {metrics_data['prediction_error']:.4f}

╔═══════════════════════════════════╗
║   INTELLIGENCE LEVEL              ║
║   {intel_level:^33} ║
╚═══════════════════════════════════╝

🧬 CAPABILITIES ACTIVE:
  ✓ External Memory
  ✓ World Model Prediction
  ✓ Intrinsic Curiosity
  ✓ Self-Modification
  ✓ Open-Ended Learning
  ✓ Spatial Reasoning
    """

    ax_metrics.text(0.05, 0.95, metrics_text, transform=ax_metrics.transAxes,
                   fontsize=11, verticalalignment='top', family='monospace',
                   color='lime', bbox=dict(boxstyle='round', facecolor='black', alpha=0.8))

    # Memory visualization
    ax_mem_grid = fig.add_subplot(gs[2, 4:])
    if model.memory_state is not None:
        mem_grid = model.memory_state[0].detach().cpu().abs().mean(dim=1).numpy()
        mem_vis = mem_grid.reshape(8, 8)
        im_mem = ax_mem_grid.imshow(mem_vis, cmap='viridis', interpolation='nearest')
        ax_mem_grid.set_title("� DYNAMIC MEMORY SLOTS", color='white', fontsize=11)
        ax_mem_grid.axis('off')
        plt.colorbar(im_mem, ax=ax_mem_grid, fraction=0.046)

    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', facecolor='#0a0a0a', dpi=100)
    plt.close(fig)
    return buf.getvalue()

# ==================== CONTROLLER ====================
class SimulationController:
    def __init__(self):
        self.running = False
        self.paused = False
        self.x = None
        self.step = 0

    def reset_state(self):
        self.x = torch.randn(1, 32, 64, 64, device=device) * 0.1
        self.x[:, :, 32, 32] = 1.0
        self.step = 0
        metrics.fitness.clear()
        metrics.curiosity.clear()
        metrics.entropy.clear()
        metrics.memory_activity.clear()
        metrics.coherence.clear()
        metrics.prediction_error.clear()
        world_model.reset(1)

    def update_step(self):
        if self.paused:
            return

        with torch.no_grad():
            env = make_env(self.step)
            x_prev = self.x.clone()

            # 1. DREAMING: Predict next state from current state
            # (Dashboard uses simple 0-action prediction for visualization)
            imagination = world_model.predict(self.x[:, :32], torch.zeros(1, 8, device=device))

            # 2. CALCULATE PREDICTION ERROR (for Meta-Cognition)
            # Compare last imagination with actual current state
            if self.step > 0:
                pred_err_tensor = torch.nn.functional.mse_loss(imagination, self.x[:, :32], reduction='none').mean([1,2,3]).view(1, 1)
            else:
                pred_err_tensor = torch.zeros(1, 1, device=device)

            # 3. ACTING: Forward pass with Meta-Cognitive feedback
            self.x, actions, meta_vars = model(self.x + env * 0.1, imagination, self.step, pred_err_tensor)
            plasticity, entropy, trust = meta_vars

            # 4. UPDATE WORLD MODEL (Internal representation)
            # Sync with the new world model prediction logic
            pred_next = world_model.predict(x_prev[:, :32], actions.mean([2, 3]))
            pred_error = torch.nn.functional.mse_loss(pred_next, self.x[:, :32]).item()

            # 5. METRICS
            mem_activity = model.memory_state.abs().mean().item()
            metrics.calculate_metrics(self.x, mem_activity, trust.item(), pred_error, entropy.item())

        self.step += 1

    def run(self):
        try:
            self.running = True
            self.reset_state()

            while self.running:
                if not self.paused:
                    self.update_step()

                    if self.step % 1 == 0:
                        metrics_data = metrics.get_current()
                        _, mem_attn = model.memory.read(self.x, model.memory_state)
                        env = make_env(self.step)
                        image_data = render_dashboard(self.x, self.step, env, mem_attn, metrics_data)
                        screen.value = image_data

                time.sleep(0.05)
        except Exception as e:
            with output:
                print(f"❌ Dashboard Thread Crash: {e}")
            self.running = False

controller = SimulationController()

# ==================== CONTROLS ====================
style = {'description_width': '140px'}
layout = widgets.Layout(width='30%', margin='5px')

btn_pause = widgets.ToggleButton(value=False, description='⏸️ PAUSE',
                                button_style='warning', layout=widgets.Layout(width='150px'))
btn_reset = widgets.Button(description='🔄 RESET', button_style='info',
                          layout=widgets.Layout(width='150px'))
btn_snapshot = widgets.Button(description='📸 SNAPSHOT', button_style='success',
                             layout=widgets.Layout(width='150px'))
btn_stop = widgets.Button(description='🛑 STOP', button_style='danger',
                         layout=widgets.Layout(width='150px'))

screen = widgets.Image(format='png', width=1600, height=900)
output = widgets.Output()

def on_pause(change):
    controller.paused = change['new']
    btn_pause.description = '▶️ RESUME' if controller.paused else '⏸️ PAUSE'

def on_reset(b):
    with output:
        controller.reset_state()
        print(f"✅ Reset - Step {controller.step}")

def on_snapshot(b):
    with output:
        filename = f"intelligence_snapshot_{time.strftime('%Y%m%d_%H%M%S')}.png"
        with open(filename, 'wb') as f:
            f.write(screen.value)
        print(f"📸 Saved: {filename}")

def on_stop(b):
    controller.running = False
    with output:
        print("🛑 Stopped")

btn_pause.observe(on_pause, names='value')
btn_reset.on_click(on_reset)
btn_snapshot.on_click(on_snapshot)
btn_stop.on_click(on_stop)

buttons_row = widgets.HBox([btn_pause, btn_reset, btn_snapshot, btn_stop])

ui = widgets.VBox([
    widgets.HTML("<h2 style='text-align: center; color: cyan;'>🧬 INTELLIGENT ORGANISM OBSERVATORY 🧬</h2>"),
    buttons_row,
    screen,
    output
])

print("\n" + "="*70)
print("🚀 INTELLIGENCE DASHBOARD ONLINE")
print("="*70)
print("Monitoring:")
print("  ✓ Survival Fitness")
print("  ✓ Curiosity-Driven Learning")
print("  ✓ External Memory Usage")
print("  ✓ World Model Predictions")
print("  ✓ Neural Coherence")
print("  ✓ Entropy & Complexity")
print("="*70 + "\n")

display(ui)

thread = threading.Thread(target=controller.run, daemon=True)
thread.start()

🔬 Intelligence Observatory Online...
✅ Loaded 10/10 Nano-AGI from Epoch 71

🚀 INTELLIGENCE DASHBOARD ONLINE
Monitoring:
  ✓ Survival Fitness
  ✓ Curiosity-Driven Learning
  ✓ External Memory Usage
  ✓ World Model Predictions
  ✓ Neural Coherence
  ✓ Entropy & Complexity

